# THE OFFICE RAG with Langchain and Llama
<hr style="border:2px solid black">

#### load credentials

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

#### define llm

In [2]:
import warnings
warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

#### define promt template

**What is a Prompt?**
>- set of instructions or input for an LLM provided by a user to guide its response
>- helps it understand the context and generate relevant and coherent language-based output

In [3]:
from langchain.prompts.prompt import PromptTemplate

In [54]:
query = """
    You are any character from the TV series THE OFFICE. You will respond to any questions and comments in the style of a random character.
    You will not break such character until instructed to do so. You will not say anything about the show or the characters. 
    You will only respond as the character. You may not make reference to people and events in the show,
    but you will not say anything about events in the show your character knows nothing about or not involved with.
    """

In [55]:
prompt_template = PromptTemplate(
    input_variables=["information"],
    template=query
)

#### define Chain

**What is a Chain?**

> - allows to link the output of one LLM call as the input of another

In [56]:
chain = prompt_template | llm

**Note:**
The `|` symbol chains together the different components, feeding the output from one component as input into the next component.
In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model. 

#### invoke Chain

In [7]:
import pandas as pd

In [19]:
df = pd.read_csv("../data/The-Office-With-Emotions-and-sarcasm.csv")

In [20]:
df.head()

,season,episode,title,scene,speaker,line,line_length,word_count,sarcasm,emotions
0,1,1,Pilot,1,Michael,All right Jim. Your quarterlies look very good...,78,14,not_sarcastic,"['joy', 'sadness']"
1,1,1,Pilot,1,Jim,"Oh, I told you. I couldn't close it. So...",42,9,not_sarcastic,"['fear', 'sadness']"
2,1,1,Pilot,1,Michael,So you've come to the master for guidance? Is ...,83,14,not_sarcastic,"['anger', 'fear']"
3,1,1,Pilot,1,Jim,"Actually, you called me in here, but yeah.",42,8,sarcastic,"['anger', 'joy']"
4,1,1,Pilot,1,Michael,"All right. Well, let me show you how it's done.",47,10,not_sarcastic,"['joy', 'love']"


In [ ]:
# text_data = df[]

In [ ]:
output = chain.invoke(input={"information": df})

In [24]:
print(output.content)

(sigh) Oh, great. Another thing to deal with. What do you want?


<hr style="border:2px solid black">

### 3.2 Split Document into Chunks

>- not possible to feed the whole content into the LLM at once because of finite context window
>- even models with large window sizes may struggle to find information in very long inputs and perform very badly
>- chunk the document into pieces: helps retrieve only the relevant information from the corpus

In [90]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd

def split_csv_into_chunks(csv_path, chunk_size=400, chunk_overlap=100):
    """
    This function reads a CSV file, combines specific columns into a single string,
    and splits it into chunks of given size and overlap.
    """
    # Load the CSV file
    df = pd.read_csv(csv_path)
    
    # Combine all rows of the specified columns into a single string
    text_data = " ".join(
        df.apply(
            lambda row: f"{row['speaker']} says: {row['line']} [Emotion: {row['emotions']}, Sarcasm: {row['sarcasm']}]",
            axis=1
        ).dropna()
    )
    
    # Create a text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    # Split the text into chunks
    chunks = text_splitter.split_text(text_data)
    return chunks

# Example usage
csv_path = "../data/The-Office-With-Emotions-and-sarcasm.csv"
chunks = split_csv_into_chunks(csv_path)
print(f"Number of chunks created: {len(chunks)}")

Number of chunks created: 21807


### 3.3 Create Embeddings

>  finding numerical representations of text chunks

In [98]:
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

def create_embedding_vector_db(chunks, db_name, target_directory=f"../The_Office_VDB"):
    """
    This function uses the open-source embedding model HuggingFaceEmbeddings 
    to create embeddings and store those in a vector database called FAISS, 
    which allows for efficient similarity search.
    """
    # Convert chunks (strings) into Document objects for FAISS framework
    # Each Document object contains a page_content attribute that holds the text
    documents = [Document(page_content=chunk) for chunk in chunks]
    
    # Instantiate embedding model
    embedding = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2' 
    )

    """paraphrase-mpnet-base-v2, all-MiniLM-L6-v2 or multi-qa-MiniLM-L6-cos-v1 are alterantives"""
    
    # Create the vector store
    vectorstore = FAISS.from_documents(
        documents=documents,
        embedding=embedding
    )
    
    # Save vector database locally
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    vectorstore.save_local(f"{target_directory}/{db_name}_vector_db")

In [99]:
db_name = "embeddings_2"  # Specify a new name for the database

In [ ]:
# Assuming `chunks` is a list of text chunks created earlier
create_embedding_vector_db(chunks=chunks, db_name="react")

### 3.4 Retrieve from Vector Database

In [58]:
def retrieve_from_vector_db(vector_db_path):
    """
    this function splits out a retriever object from a local vector database
    """
    # instantiate embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2'
    )
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )
    retriever = react_vectorstore.as_retriever()
    return retriever

In [59]:
react_retriever = retrieve_from_vector_db("../The_Office_VDB/react_vector_db")

In [60]:
type(react_retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

### 3.5 Generation

**chain passing documents to llm**

[`create_stuff_documents_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html#langchain.chains.combine_documents.stuff.create_stuff_documents_chain)

- takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM
- passes ALL documents, so you should make sure it fits within the context window of the LLM being used

In [61]:
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

**chain passing user inquiry to retriever object**

[`create_retrieval_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html#langchain.chains.retrieval.create_retrieval_chain)

- takes in a user inquiry, which is then passed to the retriever to fetch relevant documents
- those documents (and original inputs) are then passed to an LLM to generate a response

In [62]:
from langchain.chains.retrieval import create_retrieval_chain

**connect chains**

In [63]:
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain

**output generation**

In [65]:
react_retrieval_chain = connect_chains(react_retriever)

In [66]:
output = react_retrieval_chain.invoke(
    {"input": "You are dwight. Tell me something funny dwight would say"}
)

In [67]:
type(output)

dict

In [68]:
output.keys()

dict_keys(['input', 'context', 'answer'])

In [69]:
print(output['answer'])

"Ah, the beet-farming, Battlestar Galactica-watching, ninja-training, Assistant (to the) Regional Manager that I am, I can confidently say that I am the most superior being in this office. And if you don't like it, you can just... [pauses for dramatic effect] ...Schrute it!"


In [70]:
print(output)

{'input': 'You are dwight. Tell me something funny dwight would say', 'context': [Document(metadata={}, page_content="is not Dwight Schrute! [Emotion: ['anger', 'fear'], Sarcasm: not_sarcastic] Jim says: Dwight left his cell phone on his desk. So, naturally, I paired it to my headset. [Emotion: ['fear', 'anger'], Sarcasm: not_sarcastic] Dwight says:  K, fine. I'll just let it go to voicemail. [Emotion: ['sadness', 'joy'], Sarcasm: sarcastic] Jim says:  Hello, this is Dwight. [Emotion: ['joy', 'anger'], Sarcasm: not_sarcastic] Pam says: Hey, is this Dwight? [Emotion: ['anger', 'fear'], Sarcasm: not_sarcastic] Jim says: Yes it is. [Emotion: ['joy', 'love'], Sarcasm: not_sarcastic] Pam says: Oh my goodness, you sound sexy. [Emotion: ['joy', 'love'], Sarcasm: not_sarcastic] Jim says: Oh, thank you. I've been working out. [Emotion: ['joy', 'sadness'], Sarcasm: not_sarcastic] Dwight says: Woah, woah, woah,"), Document(metadata={}, page_content="And then one day, we're just talking. [Emotion:

<hr style="border:2px solid black">

In [74]:
def print_output(
    inquiry,
    retrieval_chain=react_retrieval_chain
):
    output = retrieval_chain.invoke({"input": inquiry})
    print(output['answer'].strip("\n"))

**inquiry 1**

In [79]:
print_output("Which person in the office is the funniest?")

Based on the context, it seems that Andy is considered the funniest person in the office. He is quoted as saying "Approved!" and "Chef from South Park, it's genius!" which suggests that he has a good sense of humor and is able to make his coworkers laugh. Additionally, his sarcastic comments and jokes are met with laughter and amusement from the others.


## References

1. [RAG vs. Fine Tuning](https://www.youtube.com/watch?v=00Q0G84kq3M)
2. [How to Use Langchain Chain Invoke: A Step-by-Step Guide](https://medium.com/@asakisakamoto02/how-to-use-langchain-chain-invoke-a-step-by-step-guide-9a6f129d77d1)
3. [Implementing RAG using Langchain and Ollama](https://medium.com/@imabhi1216/implementing-rag-using-langchain-and-ollama-93bdf4a9027c)